In [22]:
import sys
import os 
sys.path.append('../../../Mental_Disorder/3_feature_visualization') # get old tweets library
import age_gender_predictor
from collections import defaultdict
import re
from tabulate import tabulate
from datetime import datetime, timedelta
import math

## Emotion Ratios
Each ratio represents the percentage of an emotion in total tweets

## Load tweets & emotion

In [7]:
def listFolderFile(folder):
    return os.listdir(folder)
def loadTweets():
    # {username:{int(date):{[(datetime,content),...]}}}
    tweets_dict = defaultdict(lambda: defaultdict(lambda: []))
    folder = '../patient emotion/'
    for user_file in listFolderFile(folder):
        with open(folder + user_file) as tweets:
            for line in tweets.readlines():
                username, date, content, emotion1, emotion2, ambiguous = line.split('\t')
                tweets_dict[username][int(date)].append((content, emotion1, emotion2, ambiguous))

    return tweets_dict

In [8]:
tweets_dict = loadTweets()

## Get user ill time

In [11]:
def readPatientIllTime(folder, filename):
    with open(folder + filename, 'r') as openfile:
        return [line.strip().split('\t') for line in openfile.readlines()]



In [20]:
# ## Get ill time information
patient_ill_time_list = readPatientIllTime('../../twitter crawler/', 'bipolar_list')
patient_ill_time_dict = {line[0]: line[1] for line in patient_ill_time_list}

# dict[user][diagnosed_time(datetime)]
patient_month_time_dict = {}

for patient in patient_ill_time_dict:
    datetime_list = patient_ill_time_dict[patient].split('/')
    if len(datetime_list) > 1:
        if len(datetime_list) > 2:
            patient_month_time_dict[patient] = datetime(int(datetime_list[0]), int(datetime_list[1]), int(datetime_list[2]))
        else:
            patient_month_time_dict[patient] = datetime(int(datetime_list[0]),int(datetime_list[1]),1)
    else:
        continue
print( 'Patient Counts from month ill time:' + str(len(patient_month_time_dict)))

Patient Counts from month ill time:277


In [21]:
for patient, diagnosed_time in patient_month_time_dict.iteritems():
    print(patient)
    print(diagnosed_time)
    break

NiallBunny
2016-06-20 00:00:00


## Get emotion count
Get user's emotion count around ( 6 weeks ~ diagnosed time ~ 2 weeks)

In [44]:
def date_not_in_range(user, date):
    if date >= patient_month_time_dict[user] - timedelta(days=42) \
    and date <= patient_month_time_dict[user] + timedelta(days=14):
        return False
    else:
        return True


def getUsersEmotionsDict(users_timeSeries):
#     user -> emotion count
    user_emotion_dict = defaultdict(lambda: {"joy":0,
                                       "sadness": 0,
                                       "fear":0,
                                       "anticipation": 0,
                                       "anger":0,
                                       "trust": 0,
                                       "disgust": 0,
                                       "surprise" : 0
                                      })
    for user in users_timeSeries:
        user_tweets_count = 0
#         date, [tweet_info, tweets_info ...]
        for date, tweets_infos in users_timeSeries[user].iteritems():
            if date_not_in_range(user, datetime.strptime(str(date), "%Y%m%d")): continue
            for tweet_info in tweets_infos:                
    #             content, emotion1, emotion2, ambiguous
                if(tweet_info[3].strip() != 'yes'):
    #         only get the first emotion now
                    user_emotion_dict[user][tweet_info[1]] += 1
                    user_tweets_count += 1
    
        for emotion in user_emotion_dict[user]:
            if user_tweets_count == 0:
                user_emotion_dict[user][emotion] = 0
            else:
                user_emotion_dict[user][emotion] = float(user_emotion_dict[user][emotion]) / user_tweets_count
    return user_emotion_dict
        

In [45]:
user_emotion_dict = getUsersEmotionsDict(tweets_dict)

In [55]:
emotions = ["joy","sadness","fear", "anticipation", "anger", "trust", "disgust" ,"surprise"]

headers = ["User"] + emotions
contents = []
for uid, user in enumerate(user_emotion_dict):
    temp_list = [user]
    for emotion in emotions:
        emotion_ratio = "{}%".format(round(user_emotion_dict[user][emotion]*100, 2))
        temp_list.append(emotion_ratio)
        
    contents.append(temp_list)

print(tabulate(contents, headers=headers))

User             joy     sadness    fear    anticipation    anger    trust    disgust    surprise
---------------  ------  ---------  ------  --------------  -------  -------  ---------  ----------
yobitchmiranda   18.71%  21.58%     1.44%   17.99%          10.79%   5.04%    20.86%     3.6%
TionSavageee     6.93%   21.39%     2.71%   12.05%          23.49%   1.51%    28.92%     3.01%
sanchezpaige     7.26%   45.3%      2.99%   8.12%           17.09%   3.42%    13.25%     2.56%
C_Leex3          0.0%    36.36%     0.0%    36.36%          0.0%     0.0%     27.27%     0.0%
silencedrowns    10.38%  31.3%      5.46%   10.17%          10.01%   3.62%    21.24%     7.82%
LochnessaMarie   11.38%  36.59%     4.88%   12.2%           14.63%   3.25%    16.26%     0.81%
TommmMate        7.69%   30.77%     3.85%   26.92%          11.54%   7.69%    11.54%     0.0%
X_JuliiiiBabyyy  8.25%   26.75%     2.75%   13.25%          19.0%    4.5%     23.75%     1.75%
nicoletteblando  0.0%    28.57%     0.0%    1